In [1]:
import numpy as np
from lookup_table import CaseNum2EdgeOffset, getCaseNum
import trimesh
import os
import time
import tqdm

In [6]:
def marching_cube(thres,cells):
    # vertices use dictionary to avoid duplicate axes
    vertex_array = {}
    face_array = []
    t1 = time.time()
    # -------------------TODO------------------ 
    # compute vertices and faces
    # vertices: [N, 3]
    # faces: [M, 3], e.g. np.array([[0,1,2]]) means a triangle composed of vertices[0], vertices[1] and vertices[2]
    # for-loop is allowed to reduce difficulty
    # -------------------TODO------------------ 

    vertex_cnt = 0

    for x in tqdm.tqdm(range(cells.shape[0]-1)):
        for y in range(cells.shape[1]-1):
            for z in range(cells.shape[2]-1):
                cases = getCaseNum(x, y, z, thres, cells)
                temp_vertices = []
                for i, case_num in enumerate(cases):
                    if case_num == -1:
                        break
                    corner_1 = (x+CaseNum2EdgeOffset[case_num][0],y+CaseNum2EdgeOffset[case_num][1],z+CaseNum2EdgeOffset[case_num][2])
                    corner_2 = (x+CaseNum2EdgeOffset[case_num][3],y+CaseNum2EdgeOffset[case_num][4],z+CaseNum2EdgeOffset[case_num][5])
                    # use linear interpolation to compute the intersection point
                    val1, val2 = cells[corner_1], cells[corner_2]
                    corner_1, corner_2 = np.array(corner_1), np.array(corner_2)
                    _lambda = (thres-val1)/(val2-val1)
                    intersection = np.array(corner_1) + _lambda*np.array(corner_2-corner_1)
                    # add intersection to vertex_array
                    intersection = tuple(intersection)
                    if intersection not in vertex_array:
                        vertex_array[intersection] = vertex_cnt
                        vertex_cnt += 1
                    temp_vertices.append(vertex_array[intersection])

                    # add face to face_array
                    if i % 3 == 2:
                        assert len(temp_vertices) == 3
                        face_array.append(tuple(temp_vertices))
                        temp_vertices = []
       
    # extract vertices from dictionary

    vertex_array = list(vertex_array.items())
    vertex_array = sorted(vertex_array, key=lambda x: x[1])
    vertex_array = [x[0] for x in vertex_array]

    t2 = time.time()
    print("\nTime taken by algorithm\n"+'-'*40+"\n{} s".format(t2-t1))
    return np.array(vertex_array), np.array(face_array)

In [7]:
# reconstruct these two animals
shape_name_lst = ['spot', 'bob']
for shape_name in shape_name_lst:
    data = np.load(os.path.join('data', shape_name + '_cell.npy'))
    verts, faces = marching_cube(0, data)
    mesh = trimesh.Trimesh(vertices=verts, faces=faces)
    mesh_txt = trimesh.exchange.obj.export_obj(mesh)
    with open(os.path.join('../results', shape_name + '.obj'),"w") as fp:
        fp.write(mesh_txt)

100%|██████████| 63/63 [00:02<00:00, 31.23it/s]



Time taken by algorithm
----------------------------------------
2.029773235321045 s


100%|██████████| 63/63 [00:02<00:00, 30.48it/s]


Time taken by algorithm
----------------------------------------
2.069071054458618 s
